# Препроцессинг сцен Ландсат

В этом документе рассматривается схема нормализации сцен Ландстат. В основу документа положена статья Vicente-Serrano S. M., Pérez-Cabello F., Lasanta T. Assessment of radiometric correction techniques in analyzing vegetation variability and change using time series of Landsat images //Remote Sensing of Environment. – 2008. – Т. 112. – №. 10. – С. 3916-3934.

Поэтому вначале документа приводится краткий пересказ основных положений статьи, а затем реализуются ее основные методы.

## Содержимое работы

Авторы цитируемой статьи исследуют влияние различных методов преобразования данных (геометрических/радиометрических/топографических/атмосферных коррекций) на качество временных серий сцен Ландстат, полученных в результате данных преобразований.

Авторы сравнивали результаты, преоразованные последовательностью следующих коррекций:

1. Геометрическая коррекция. Поскольку сцены Ландсат распространяются уже скорректированными, то авторы не могли влиять на конкретный вид коррекции и пользовались уже готовыми данными.
2. Расчет излучения на верхней границе атмосферы. Данные Ландсат из Digial Numbers (DN) преобразовывались в величины излучения, зафиксированного сенсором на верней границе атмосферы. (Это стандартная операция, описанная во множестве источников, и вряд ли она требует дополнительного пояснения).
3. Атмосферная коррекция. Авторы рассматривали два варианта атмосферной коррекции: Dark Target Approach (DTA) и Radiative Transfer Model. Это также довольно известные методы, первый - модификация DOS метода, а второй использует код алгоритма Second Simulation of the Satellite Signal in the Solar Spectrum (6S).
4. Топографическая коррекция. Авторы рассматривали два вида топографической коррекции: (а) на основе модели Ламберта и (б) с учетом неламбертовых эффектов.
5. Относительная нормализация. Авторы рассматривали влияние относительной нормализации, когда яркости снимка регрессионными методами притягивались к яркостям с заданными средними и дисперсиями. Такими яркостями могут быть яркости пикселей второго снимка, взятого в качестве основного, или же некие абстрактные единицы (в случае, когда все снимки оказываются "притянутыми" к единому масштабу, точка отсчета становится не так важна). Авторы исследовали две реализации этой процедуры: ручной выбор опорных точек (псевдоинвариантных объектов, PIF) и автоматический на базе метода главных компонент.

*В статье довольно подробно рассматривается суть каждого метода, но в этом документы мы опустим лишние детали, при необходимости - см. исходник и ссылки на литературу в нем.*

Для оценки влияния (положительного или отрицательного) каждого этапа коррекции (тем или иным методом) авторы выбрали набор точек с заведомо не изменяющимися во времени характеристиками отраженного излучения (псевдоинвариантные объекты, PIF). Это были точки различных типов (бетон, асфальт, вода, определенные типы лесного покрова и др.). Затем авторы построили временные ряды сцен Ландсат, преобразованных выбранной последовательностью методов, и оценили изменчивость полученной отражательной способности в выбранных точках. Помимо того, авторы использовали также наземные измерения свойств объектов спектрорадиометром.

С определенными оговорками, что результаты применимы не всегда и не везде, был сделан вывод, что атмосферная коррекция не так уж и важна для получения результата, особенно если в дальнейшем будет использоваться относительная нормализация. В то же время топографическая коррекция, как ламбертова, так и неламбертова, позволяет улучшить результат.

## Детали реализации

Общая схема работы будет совпадать с методикой описанной в вышеуказанной статье. Правда, есть и различия, вытекающие из различающихся целей статьи и нашей работы. Поэтому следует оговориться, что:

* Цель статьи - сравнить результаты, полученные разными способами преобразования данных. Наша цель - преобразовать. Поэтому мы будем использовать наиболее перспективные методы, не будем реализовавать всё описанное в статье.
* В то же время не всегда понятно, какой метод даст результат лучше, а какой хуже. Поэтому в случае развилок, возможно, придется опробовать разные варианты.
* Для того, чтобы оценить качество предобработки данных нужно иметь набор точкек с заведомо неизменными во времени спектральными характеристиками. Наличие такого слоя позволит объективно сравнивать результаты, полученные разными методами (и, что не менее важно! - автоматизировать процедуру выбора оптимального набора преобразований).

Таким образом предобработка данных будет произведена по схеме, заимствованной из указанной статьи, но с определенными модификациями: 

1. Скачанные данные уже прошли геометрическую коррекцию, поэтому данный шаг опускается.
2. Шаг номер два данной методики реализовано [в блокноте с описанием инициализации данных](000_Initialization.ipynb). В нем расчет излучения на верхней границе атмосферы проделывается на этапе импорта данных Landsat.
3. В статье никак не обрабатывалась облачность (для их территории и сезона облака редки), но в нашем случае мы не можем игнорировать облачность. Поэтому необходимо ввести шаг для маскирования облаков и полностью исключить участки, покрытые облаками из анализа. Поиск и маскирование облачности следует сделать до топографической коррекции и нормализации, а также (вероятно) до атмосферной коррекции (о ней см. ниже). Вопрос маскирования решается в [блокноте](005_Preprocessing.ipynb).
4. В нашей работе планируется производить относительную нормализацию снимков, поэтому шаг с атмосферной коррекцией можно пропустить. (Имеющиеся у нас данные не позволять произвести более точную атмосферную коррекцию по алгоритму 6S, а атмосферная коррекция на базе DOS все равно будет линейной, что нивелируется относительной нормализацией. Возможно, впоследствии придейтся вернуться к вопросу об атмосферной коррекции, но на данном этапе представляется, что этот шаг излишен).
5. Топографическая коррекция реализована [в отдельном блокноте](006_TopoCorrection.ipynb).
6. Относительная нормализация данных рассматривается [в блокноте](007_Normalization.ipynb).